In [20]:
#pip install langchain-community==0.2.15  langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 "transformers>=4.41.0"faiss-cpu langchain-groq==0.1.9 PyPDF2-3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import os
from langchain.prompts import PromptTemplate
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.chat_history import BaseChatMessageHistory
import requests
import google.generativeai as genai
from langchain_core.language_models import LLM
from typing import Optional, List
from PyPDF2 import PdfFileReader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
import time
import json
from typing import List, Optional

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key="API_KEY",
    model_name="llama-3.1-8b-instant"
)


In [3]:
chain = load_qa_chain(llm, chain_type="refine")

C:\Users\acer\AppData\Local\Temp\ipykernel_27668\4254714676.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm, chain_type="refine")


In [3]:
# FunctioI to load PDF and create a retriever
def load_pdf(file):
    loader = PyPDFLoader(file)
    docs = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=20)
    texts = text_splitter.split_documents(docs)
    db_FAISS = FAISS.from_documents(texts, embeddings)
    return db_FAISS

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
db_faiss = load_pdf(r"C:\Users\acer\Downloads\FaisalRasheed_GenAI [Autosaved].pdf")
db_faiss

In [12]:
chain = load_qa_chain(llm, chain_type="refine")

In [13]:
# Function to handle SimilaritySearch
def SimilaritySearch(docs_db, user_input):
    response = chain({"input_documents": docs_db, "question": user_input, "language": "English", "existing_answer" : ""}, return_only_outputs=True)
    return response['output_text']

In [14]:
#user_input = "What is the greenhouse effect?"
user_input = "Tell me about this PDF?"

In [15]:
docs_db = db_faiss.similarity_search(user_input)
SimilaritySearch(docs_db, user_input)

C:\Users\acer\AppData\Local\Temp\ipykernel_27668\3659431881.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain({"input_documents": docs_db, "question": user_input, "language": "English", "existing_answer" : ""}, return_only_outputs=True)


"Based on the provided context, it appears that the PDF is a project report or document related to a Leave Management System chatbot developed by an ML Engineer at Capgemini (working with Google). The document outlines the key responsibilities and technologies used in the project.\n\nHere's a refined version of the original answer:\n\nThe PDF appears to be a project document detailing the development of a Leave Management System chatbot using Google DialogFlow CX Platform. The document highlights the key responsibilities of the ML Engineer, including creating an end-to-end chatbot, writing conditional codes for date validation, and using various technologies such as Google Cloud Functions, Google Firestore, and DialogFlow CX.\n\nThe document describes the system's functionality, which includes:\n\n1. Asking for employee details and providing options for various types of leave.\n2. Checking whether the leave is correct or not and requesting re-entry if necessary.\n3. Asking whether the 

In [17]:
def retrievalqa(db_faiss, user_input):
    custom_prompt_template = """
    <s>[INST] answer the questions on the given context after analysing entire content shared with you\
    If it is not found or mentioned in the given context, just return "null". \
    Do not hallucinate and don't give any additional information or explanation from your end.\
    Provide a detailed answer relevant to the prompt. \
    If answer not found just return Null. \
    If context not found return Null.\

    CONTEXT:
    {context}

    QUESTION: {question} [/INST]</s>
    I
    Answer:"""

    Ret_PROMPT = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])
    func = RetrievalQA.from_chain_type(
        llm=llm,
        #chain_type='stuff',
        retriever=db_faiss.as_retriever(),
        #chain_type_kwargs={'prompt': Ret_PROMPT}
    )

    result = func.invoke({"query": user_input})
    return result


In [18]:
db_faiss = load_pdf(r"C:\Users\acer\Downloads\FaisalRasheed_GenAI [Autosaved].pdf")
user_input = "Tell me about this PDF?"


In [19]:
retrievalqa(db_faiss, user_input)


{'query': 'Tell me about this PDF?',
 'result': "There is no PDF mentioned in the provided context. The context appears to be a collection of project descriptions, a resume, and various technologies and skills listed. If you could provide more information or clarify which PDF you are referring to, I'll do my best to assist you."}